## Simulation of the Alpha Arm with Hydrodynamics

This simulation has only the Alpha arm implemented. The URDF called, ```alphaArmCombo.urdf```, includes the added mass as inertia. 

#### Metadata

Last edited by Hannah Kolano on 12/15/2021

In [2]:
# Module importing
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [3]:
# model definition
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmCombo.urdf")
mechanism_alpha = parse_urdf(urdf_file; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file), vis[:alpha])

# Name the joints and bodies of the mechanism
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)

# Define default frames for each body
shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# setelement!(mvis_alpha, shoulder_frame) # shows the frame in the visualization

println("URDFs parsed. Visualizers ready. \n")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


URDFs parsed. Visualizers ready. 



In [5]:
# buoyancy & gravity setup
# ------------------------------------------------------------------------
#                  BUOYANCY & GRAVITY FORCE MAGNITUDES
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
volumes = [.018, .203, .025, .155] #  shoulder, ua, elbow, wrist,
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end
print(buoy_lin_forces)

masses = [.194, .429, .115, .333]
grav_forces = masses*9.81
grav_lin_forces = []
for f_g in grav_forces
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, -f_g])
    push!(grav_lin_forces, lin_force)
end

Any[

FreeVector3D in "world": [0.0, 0.0, 0.17605026], FreeVector3D in "world": [0.0, 0.0, 1.9854557100000003], FreeVector3D in "world": [0.0, 0.0, 0.24451425000000004], FreeVector3D in "world": [0.0, 0.0, 1.5159883500000002]]

In [6]:
# Hydrodynamics calculator function
# ------------------------------------------------------------------------
#                HYDRODYNAMICS (Grav & Buoy) CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    names = ["cob1", "cob2", "cob3", "cob4"]
    # Iterate through buoyancy for each body
    for i in 1:4
        # Get the body
        bod = bodies(mechanism_alpha)[i+1]
        # Get default frame of te body
        body_default_frame = default_frame(bod)
        # Get transform between the defualt frame and the center of buoyancy
        # TODO: don't use fixed_transform because it's bad for computation time
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        def_to_com = fixed_transform(bod, body_default_frame, com_frames[i])
        # Transform buoyancy force vector to the body's default frame (rotation only)
        buoy_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        grav_force_trans = transform(state_alpha, grav_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        buoy_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), buoy_force_trans)
        grav_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_com))), grav_force_trans)
        # Add the buoyancy wrench and grav wrench together
        wrench = buoy_wrench + grav_wrench
        # println(wrench)
        # setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_cob))), 0.02, "name")
        # Transform the wrench to the root frame and assign it to the body
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;


In [52]:
function get_des_acc!(des_acc, t, state_alpha::MechanismState, vdot)
    # TODO: implement a simple pt a to pt b trajectory generation
    des_acc[base_joint][1] = 0.0
    des_acc[shoulder_joint][1] = vdot[shoulder_joint][1]
    des_acc[elbow_joint][1] = 0.5 #vdot[elbow_joint][1]
    des_acc[wrist_joint][1] = 0.00
end;

In [53]:
# Controller function
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState, hydro_wrenches::Dict{BodyID, Wrench{Float64}}, vdot)

    # Calculate inverse dynamics of alpha arm (including buoyancy)
    get_des_acc!(des_acc, t, state_alpha, vdot)
    tau = inverse_dynamics(state_alpha, des_acc, hydro_wrenches)

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= 0 #tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]

end;

In [10]:
# initial conditions
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
set_configuration!(state_alpha, base_joint, 3.141)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Define desired velocities
des_acc = similar(velocity(state_alpha))
des_acc[base_joint][1] = 0.0
des_acc[shoulder_joint][1] = 0.0
des_acc[elbow_joint][1] = 0.0
des_acc[wrist_joint][1] = 0.0
# print(des_acc)


0.0

In [59]:
# simulation
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 5
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-3)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

Simulation finished.


In [60]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 0.5)
print(last(qs))

println("\n done.")

[NaN, NaN, NaN, NaN]
 done.
